Click [here](https://medium.com/@morihosseini/python-argparse-crafting-elegant-command-line-interfaces-21e7c40ccc91) to access the associated Medium article.

# Setup

In [2]:
import argparse

# Basic usage

## Positional Arguments

In [85]:
parser = argparse.ArgumentParser(
    prog="awesome", description="My Awesome CLI Tool"
)
parser.add_argument("input_file", help="Path to the input file")
parser.add_argument("output_file", help="Path to the output file")

args = parser.parse_args(["input.txt", "output.md"])
print("$ python my_script.py input.txt output.md\n")
print("input_file: ", args.input_file)
print("output_file:", args.output_file)

$ python my_script.py input.txt output.md

input_file:  input.txt
output_file: output.md


## Optional Arguments

In [116]:
parser = argparse.ArgumentParser(
    prog="awesome",
    description="My Awesome CLI Tool.",
    epilog="Example: $ python awesome.py --verbose --format=csv",
)
parser.add_argument(
    "--verbose", action="store_true", help="Enable verbose mode"
)
parser.add_argument(
    "--format", choices=["json", "csv"], default="json", help="Output format"
)

parser.print_help()

usage: awesome [-h] [--verbose] [--format {json,csv}]

My Awesome CLI Tool.

options:
  -h, --help           show this help message and exit
  --verbose            Enable verbose mode
  --format {json,csv}  Output format

Example: $ python awesome.py --verbose --format=csv


## Default Values

In [102]:
parser = argparse.ArgumentParser(prog="awesome")
parser.add_argument(
    "--threshold", type=float, default=0.5, help="Threshold value"
)

args = parser.parse_args([])
print(args.threshold)

0.5


# Advanced usage

## Subparsers

In [120]:
# Create the main parser
parser = argparse.ArgumentParser(
    prog="awesome", description="My Awesome CLI Tool"
)

# Create subparsers
subparsers = parser.add_subparsers(title="Subcommands", dest="subcommand")

# Create a subparser for the "train" command
train_parser = subparsers.add_parser(
    "train",
    help="Train a machine learning model",
    epilog="Example: "
    "$ python awesome.py train --data=data.csv --model=model.pkl"
)
train_parser.add_argument("--data", help="Path to training data")
train_parser.add_argument("--model", help="Path to save the trained model")

# Create a subparser for the "predict" command
predict_parser = subparsers.add_parser(
    "predict",
    help="Make predictions using a trained model",
    epilog="Example: "
    "$ python awesome.py predict --input=data.csv --output=pred.csv"
)
predict_parser.add_argument("--input", help="Path to input data")
predict_parser.add_argument("--output", help="Path to save predictions")

print(":::: Main parser help ::::::::::::::::::")
parser.print_help()

print("\n\n:::: Train parser help ::::::::::::::::::")
train_parser.print_help()

print("\n\n:::: Predict parser help ::::::::::::::::::")
predict_parser.print_help()

:::: Main parser help ::::::::::::::::::
usage: awesome [-h] {train,predict} ...

My Awesome CLI Tool

options:
  -h, --help       show this help message and exit

Subcommands:
  {train,predict}
    train          Train a machine learning model
    predict        Make predictions using a trained model


:::: Train parser help ::::::::::::::::::
usage: awesome train [-h] [--data DATA] [--model MODEL]

options:
  -h, --help     show this help message and exit
  --data DATA    Path to training data
  --model MODEL  Path to save the trained model

Example: $ python awesome.py train --data=data.csv --model=model.pkl


:::: Predict parser help ::::::::::::::::::
usage: awesome predict [-h] [--input INPUT] [--output OUTPUT]

options:
  -h, --help       show this help message and exit
  --input INPUT    Path to input data
  --output OUTPUT  Path to save predictions

Example: $ python awesome.py predict --input=data.csv --output=pred.csv


## Custom Argument Types

In [150]:
def valid_positive_int(value):
    try:
        int_value = int(value)
        if int_value <= 0:
            print(f"Error: {value} is not a positive integer")
        return int_value
    except ValueError:
        print(f"Error: {value} is not a valid integer")


parser = argparse.ArgumentParser(prog="awesome")
parser.add_argument("--epochs", type=valid_positive_int, default=10)

print("$ python awesome.py --epochs=-2\n")
args = parser.parse_args(["--epochs=-2"])

$ python awesome.py --epochs=-2

Error: -2 is not a positive integer


## Argument Groups

In [170]:
parser = argparse.ArgumentParser(
    prog="awesome",
    epilog="Example: $ python awesome.py --hidden_units=64 --epochs=20",
)

group = parser.add_argument_group("Model Configuration")
group.add_argument(
    "--hidden_units", type=int, default=128, help="Number of hidden units"
)
group.add_argument(
    "--learning_rate", type=float, default=0.001, help="Learning rate"
)

group = parser.add_argument_group("Training Configuration")
group.add_argument("--batch_size", type=int, default=32, help="Batch size")
group.add_argument("--epochs", type=int, default=10, help="Number of epochs")

parser.print_help()

usage: awesome [-h] [--hidden_units HIDDEN_UNITS]
               [--learning_rate LEARNING_RATE] [--batch_size BATCH_SIZE]
               [--epochs EPOCHS]

options:
  -h, --help            show this help message and exit

Model Configuration:
  --hidden_units HIDDEN_UNITS
                        Number of hidden units
  --learning_rate LEARNING_RATE
                        Learning rate

Training Configuration:
  --batch_size BATCH_SIZE
                        Batch size
  --epochs EPOCHS       Number of epochs

Example: $ python awesome.py --hidden_units=64 --epochs=20


## Mutually Exclusive Arguments

In [178]:
parser = argparse.ArgumentParser(prog="awesome")
exclusive_group = parser.add_mutually_exclusive_group()
exclusive_group.add_argument(
    "--train", action="store_true", help="Train the model"
)
exclusive_group.add_argument(
    "--predict", action="store_true", help="Make predictions"
)

try:
    # $ python awesome.py --train --predict
    parser.parse_args(["--train", "--predict"])
except SystemExit:
    pass

usage: awesome [-h] [--train | --predict]
awesome: error: argument --predict: not allowed with argument --train


# Best Practices

## Descriptive Help Messages

In [179]:
parser = argparse.ArgumentParser(prog="awesome")
parser.add_argument("--input", help="Path to input data (required)")
parser.add_argument("--output", help="Path to save results (optional)")

parser.print_help()

usage: awesome [-h] [--input INPUT] [--output OUTPUT]

options:
  -h, --help       show this help message and exit
  --input INPUT    Path to input data (required)
  --output OUTPUT  Path to save results (optional)


## Validating Input with `choices`

In [184]:
parser = argparse.ArgumentParser(prog="awesome")
parser.add_argument(
    "--format", choices=["json", "csv"], default="json", help="Output format"
)

try:
    # $ python awesome.py --format=tsv
    parser.parse_args(["--format=tsv"])
except SystemExit:
    pass

usage: awesome [-h] [--format {json,csv}]
awesome: error: argument --format: invalid choice: 'tsv' (choose from 'json', 'csv')


## Using `metavar` for User-Friendly Argument Names

In [185]:
parser = argparse.ArgumentParser(prog="awesome")
parser.add_argument("--input", metavar="data_file", help="Path to input data")

parser.print_help()

usage: awesome [-h] [--input data_file]

options:
  -h, --help         show this help message and exit
  --input data_file  Path to input data


## File Input/Output

In [202]:
parser = argparse.ArgumentParser(prog="awesome")
parser.add_argument("--input", type=argparse.FileType("r"), help="Input file")
parser.add_argument(
    "--output", type=argparse.FileType("w"), help="Output file"
)

parser.print_help()

# Uncomment the following lines to write to output.txt
# args = parser.parse_args(["--output=output.txt"])
# args.output.write("Hello, World!\n")

usage: awesome [-h] [--input INPUT] [--output OUTPUT]

options:
  -h, --help       show this help message and exit
  --input INPUT    Input file
  --output OUTPUT  Output file


## Documenting Your CLI Tool

In [204]:
parser = argparse.ArgumentParser(
    prog="awesome", description="My Awesome CLI Tool: Perform magic with data!"
)

parser.print_help()

usage: awesome [-h]

My Awesome CLI Tool: Perform magic with data!

options:
  -h, --help  show this help message and exit
